In [ ]:
import abc
import logging
from datetime import UTC, datetime
from enum import StrEnum
from pathlib import Path
from typing import Annotated, ClassVar, Literal

import entitysdk
from pydantic import Field, NonNegativeFloat, PositiveFloat, PrivateAttr

from obi_one.core.block import Block
from obi_one.core.exception import OBIONEError
from obi_one.core.info import Info
from obi_one.core.scan_config import ScanConfig
from obi_one.core.single import SingleConfigMixin
from obi_one.scientific.from_id.circuit_from_id import (
    CircuitFromID,
    MEModelWithSynapsesCircuitFromID,
)
from obi_one.scientific.from_id.memodel_from_id import MEModelFromID
from obi_one.scientific.library.circuit import Circuit
from obi_one.scientific.library.constants import (
    _COORDINATE_CONFIG_FILENAME,
    _DEFAULT_SIMULATION_LENGTH_MILLISECONDS,
    _MAX_SIMULATION_LENGTH_MILLISECONDS,
    _MIN_SIMULATION_LENGTH_MILLISECONDS,
    _SCAN_CONFIG_FILENAME,
)
from obi_one.scientific.library.memodel_circuit import MEModelCircuit, MEModelWithSynapsesCircuit
from obi_one.scientific.unions.unions_manipulations import (
    SynapticManipulationsReference,
    SynapticManipulationsUnion,
)
from obi_one.scientific.unions.unions_neuron_sets import (
    MEModelWithSynapsesNeuronSetUnion,
    NeuronSetReference,
    SimulationNeuronSetUnion,
)
from obi_one.scientific.unions.unions_recordings import (
    RecordingReference,
    RecordingUnion,
)
from obi_one.scientific.unions.unions_stimuli import (
    MEModelStimulusUnion,
    StimulusReference,
    StimulusUnion,
)
from obi_one.scientific.unions.unions_timestamps import (
    TimestampsReference,
    TimestampsUnion,
)

L = logging.getLogger(__name__)


DEFAULT_NODE_SET_NAME = "Default: All Biophysical Neurons"
DEFAULT_TIMESTAMPS_NAME = "Default: Simulation Start (0 ms)"


In [ ]:
from obi_one.core.scan_config import ScanConfig

from obi_one.core.block_reference import BlockReference
from obi_one.scientific.unions.unions_neuron_sets import NeuronSetUnion
from obi_one.scientific.unions.unions_timestamps import TimestampsUnion

class NeuronSetOrTimestampsReference(BlockReference):
    allowed_block_types = NeuronSetUnion | TimestampsUnion

class TestBlock(Block):

    test_param: NeuronSetReference | None = Field(
        reference_type=NeuronSetReference.__name__,
        default=None,
    )

class TestScanConfig(ScanConfig):
    neuron_sets: dict[str, SimulationNeuronSetUnion] = Field(
        default_factory=dict,
        reference_type=NeuronSetReference.__name__,
    )

    timestamps: dict[str, TimestampsUnion] = Field(
        default_factory=dict,
        reference_type=TimestampsReference.__name__,
    )

    test_block: TestBlock



In [ ]:
import obi_one as obi
test_conf = TestScanConfig.empty_config()

sim_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(5)))
test_conf.add(sim_neuron_set, name='L1All')

regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=10)
test_conf.add(regular_timestamps, name='RegularTimestamps')

test_block = TestBlock(test_param=sim_neuron_set.ref)
test_conf.set(test_block, name='test_block')

validated_test_conf = test_conf.validated_config()